In [ ]:
# 1. Select a pool of large, liquid, optionable, dividend-yielding stocks
# 2. Collect the expected dividend and ex-dividend dates of these stocks
# 3. Select an options month that is after the ex-dividend date
# 4. Select an OTM option that provides a balance of liquidity, upside, and credit

In [230]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import OrderedDict
import pandas as pd

url_base = "https://finviz.com/screener.ashx?"
url_params = OrderedDict([
    ("v", "161"),
    ("f", "cap_mega,fa_payoutratio_pos,idk_sp500"),
    ("ft", "4"),
    ("o", "-dividendyield"),
])
screener_url = url_base + "&".join(["{}={}".format(k, url_params[k]) for k in url_params])

def _yield_contentrows(contenttable):
    contentrows = contenttable.findAll('tr')
    headers = [td.text for td in contentrows[0].findAll('td', recursive=False)]
    for contentrow in contentrows[1:]:
        values = [td.text for td in contentrow.findAll('td')]
        yield OrderedDict([(header, value) for header, value in zip(headers, values)])

print("Loading: {}".format(screener_url))
with urlopen(screener_url) as webobj:
    soup = BeautifulSoup(webobj.read(), "lxml")
    
content = soup.find('div', {'id': 'screener-content'})
subtable = content.find('table')
subrows = subtable.findAll('tr', recursive=False)
contentrow = subrows[3]
contenttable = contentrow.find('table')
screener_df = pd.DataFrame(_yield_contentrows(contenttable))
screener_df = screener_df.set_index("Ticker")
    
screener_df

Loading: https://finviz.com/screener.ashx?v=161&f=cap_mega,fa_payoutratio_pos,idk_sp500&ft=4&o=-dividendyield


,Change,Curr R,Debt/Eq,Dividend,Earnings,Gross M,LTDebt/Eq,Market Cap,No.,Oper M,Price,Profit M,Quick R,ROA,ROE,ROI,Volume
Ticker,,,,,,,,,,,,,,,,,
T,0.30%,0.80,1.04,5.93%,Oct 23/a,51.60%,0.92,245.46B,1,12.30%,33.72,20.10%,0.80,6.80%,21.40%,5.00%,"33,644,591"
RDS-A,1.91%,1.30,0.41,5.70%,Oct 25/b,17.80%,0.36,275.37B,2,7.20%,66.02,5.70%,0.90,4.80%,10.10%,4.30%,"3,134,687"
VZ,-0.57%,1.00,2.20,4.43%,Oct 23/b,58.00%,2.10,226.34B,3,19.00%,54.39,23.90%,0.90,11.90%,71.70%,11.60%,"13,394,157"
XOM,0.26%,0.80,0.22,3.92%,Oct 26/b,32.70%,0.11,357.18B,4,6.00%,83.63,8.30%,0.50,6.00%,11.20%,3.20%,"8,831,357"
CVX,0.53%,1.10,0.25,3.79%,Oct 26/b,41.70%,0.20,224.27B,5,7.00%,118.13,8.20%,0.90,4.70%,8.10%,1.70%,"4,767,151"
NVS,-0.12%,1.10,0.42,3.50%,Oct 17/b,67.00%,0.30,215.36B,6,29.00%,85.03,26.50%,0.90,10.20%,18.80%,7.10%,"1,878,984"
PG,-0.30%,0.80,0.61,3.42%,Jul 31/b,50.00%,0.41,210.85B,7,20.50%,84.00,14.20%,0.70,7.70%,17.90%,13.10%,"5,925,207"
WFC,0.35%,-,1.22,3.16%,Oct 12/b,-,1.22,263.77B,8,77.50%,54.50,32.00%,-,1.00%,10.90%,9.80%,"21,841,850"
PFE,1.16%,1.20,0.58,3.13%,Jul 31/b,78.90%,0.41,255.23B,9,25.40%,43.51,42.30%,0.90,13.40%,33.20%,11.00%,"19,248,793"


In [234]:
for ticker in screener_df.index:
    nasdaq_url = "https://www.nasdaq.com/symbol/{}/dividend-history".format(ticker.lower().replace("-", "."))
    
    print("Loading: {}".format(nasdaq_url))  
    with urlopen(nasdaq_url) as webobj:
        soup = BeautifulSoup(webobj.read(), "lxml")
    
#     genTable = soup.find("div", {"class": "genTable"})
#     first_row = genTable.tbody.tr
#     dt = first_row.td.text.strip()
    
    
    exdate_td = soup.find("span", id="quotes_content_left_dividendhistoryGrid_exdate_0")
    exdate = pd.to_datetime(exdate_td.text.strip())
    
    dividend_td = exdate_td.find_next("span", id="quotes_content_left_dividendhistoryGrid_CashAmount_0")
    dividend = float(dividend_td.text.strip())
    screener_df.loc[ticker, "Ex-Div Date"] = exdate
    screener_df.loc[ticker, "Dividend Quarterly Cash"] = dividend

Loading: https://www.nasdaq.com/symbol/t/dividend-history
Loading: https://www.nasdaq.com/symbol/rds.a/dividend-history
Loading: https://www.nasdaq.com/symbol/vz/dividend-history
Loading: https://www.nasdaq.com/symbol/xom/dividend-history
Loading: https://www.nasdaq.com/symbol/cvx/dividend-history
Loading: https://www.nasdaq.com/symbol/nvs/dividend-history
Loading: https://www.nasdaq.com/symbol/pg/dividend-history
Loading: https://www.nasdaq.com/symbol/wfc/dividend-history
Loading: https://www.nasdaq.com/symbol/pfe/dividend-history
Loading: https://www.nasdaq.com/symbol/intc/dividend-history
Loading: https://www.nasdaq.com/symbol/jnj/dividend-history
Loading: https://www.nasdaq.com/symbol/wmt/dividend-history
Loading: https://www.nasdaq.com/symbol/bac/dividend-history
Loading: https://www.nasdaq.com/symbol/jpm/dividend-history
Loading: https://www.nasdaq.com/symbol/hd/dividend-history
Loading: https://www.nasdaq.com/symbol/ba/dividend-history
Loading: https://www.nasdaq.com/symbol/msft

In [245]:
import datetime
from dateutil.relativedelta import relativedelta

def estimate_next(dt):
    if dt < datetime.datetime.today():
        dt += relativedelta(months=3)
    if dt < datetime.datetime.today():
        print("Warning: something is funky with dt: {}".format(dt))
        dt += relativedelta(months=9)
    return dt
    
screener_df["Ex-Div Date Est."] = screener_df["Ex-Div Date"].apply(estimate_next)
print(screener_df["Ex-Div Date Est."])

Ticker
T       2018-10-09
RDS-A   2018-11-09
VZ      2018-10-09
XOM     2018-11-10
CVX     2018-11-16
NVS     2019-03-06
PG      2018-10-19
WFC     2018-11-09
PFE     2018-11-02
INTC    2018-11-06
JNJ     2018-11-27
WMT     2018-12-06
BAC     2018-12-06
JPM     2018-10-05
HD      2018-11-29
BA      2018-11-09
MSFT    2018-11-15
UNH     2018-12-06
AAPL    2018-11-10
V       2018-11-16
Name: Ex-Div Date Est., dtype: datetime64[ns]


In [246]:
def fix_year(dt):
    today = datetime.date.today()
    dt = datetime.date(year=today.year, month=dt.month, day=dt.day)
    if dt < today:
        plusq = dt + datetime.timedelta(days=92)
        
        if plusq < today:
            return dt + datetime.timedelta(days=365)
        else:
            return plusq
    else:
        return dt

def clean_earnings(ed_raw):
    ed_date_raw, ed_time_raw = ed_raw.split("/")
    ed_date = datetime.datetime.strptime(ed_date_raw, "%b %d")
    ed_date = fix_year(ed_date)
    return ed_date

screener_df["Earnings Est."] = screener_df["Earnings"].apply(clean_earnings)

print(screener_df["Earnings Est."])

Ticker
T        2018-10-23
RDS-A    2018-10-25
VZ       2018-10-23
XOM      2018-10-26
CVX      2018-10-26
NVS      2018-10-17
PG       2018-10-31
WFC      2018-10-12
PFE      2018-10-31
INTC     2018-10-25
JNJ      2018-10-16
WMT      2018-11-16
BAC      2018-10-15
JPM      2018-10-12
HD       2018-11-14
BA       2018-10-24
MSFT     2018-10-18
UNH      2018-10-16
AAPL     2018-10-31
V        2018-10-24
Name: Earnings Est., dtype: object


In [247]:
def load_expirations(ticker):
    clean_ticker = ticker.replace("-", ".").lower()
    url = "https://www.nasdaq.com/symbol/{}/option-chain".format(clean_ticker)
    
    print("Loading: {}".format(url))
    with urlopen(url) as webobj:
        soup = BeautifulSoup(webobj.read(), "lxml")
    
    chain_dates = soup.find("div", id="OptionsChain-dates")
    
    for a in chain_dates.find_all("a"):
        s = a.text
        try:
            dt = datetime.datetime.strptime(s, "%b %y")
            dt = datetime.date(year=dt.year, month=dt.month, day=1)
            yield (dt, a.attrs["href"])
        except ValueError:
            pass

def get_expiration_links(ticker):
    return pd.DataFrame(load_expirations(ticker), columns=("exp", "link"))


In [248]:
def parse_table(soup):
    chain = soup.find("div", {"class": "OptionsChain-chart"})
    header = chain.table.thead
    body = chain.table
    header_links = header.tr.find_all("th")
    columns = [link.text.split()[0].strip() for link in header_links]
    print("Parsed columns: {}".format(columns))
        
    for entry in body.find_all("tr")[1:]:
        tds = entry.find_all("td")
        rv = OrderedDict()
        mode = None
        for header, td in zip(columns, tds):
            if header in ["Calls", "Puts"]:
                mode = header
                link = td.a
                if link:
                    rv[header] = link.text
                    rv[header + "_link"] = td.a.attrs["href"]
                else:
                    rv[header] = td.text
                    rv[header + "_link"] = None
            elif header == "Root":
                rv[header] = td.text
            elif header == "Strike":
                rv[header] = float(td.text)
            else:
                pheader = mode + "_" + header
                try:
                    rv[pheader] = float(td.text)
                except ValueError:
                    rv[pheader] = None
                
        yield rv

In [249]:
def get_best_option(ticker, min_strike):
    exp_links = get_expiration_links(ticker)
    print(exp_links)

    dividend_date = screener_df.loc[ticker, "Ex-Div Date Est."]
    best_exp = exp_links[exp_links.exp > dividend_date.date()].iloc[0]
    print("Selected: {}".format(best_exp.exp))

    url = best_exp.link
    print("Loading: {}".format(url))
    with urlopen(url) as webobj:
        soup = BeautifulSoup(webobj.read(), "lxml")

    chain_df = pd.DataFrame(parse_table(soup))

    otm_calls = chain_df[chain_df.Strike >= min_strike]
    print("Found {} OTM calls".format(len(otm_calls.index)))
    max_open = otm_calls.Calls_Open.argmax()
    highest_open = otm_calls.loc[max_open]
    without_put = highest_open[[k for k in otm_calls.columns if "Puts" not in k]]
    
    return without_put

In [250]:
for ticker in screener_df.index:
    price = float(screener_df.loc[ticker, "Price"])
    best_option = get_best_option(ticker, price)
    print("Best option for {}: {}".format(ticker, best_option))
    screener_df.loc[ticker, "Call_Strike"] = float(best_option.Strike)
    screener_df.loc[ticker, "Call_Expiration"] = pd.to_datetime(best_option.Calls)
    screener_df.loc[ticker, "Call_OpenInt"] = int(best_option.Calls_Open)
    screener_df.loc[ticker, "Call_PriceLast"] = float(best_option.Calls_Last)

screener_df

Loading: https://www.nasdaq.com/symbol/t/option-chain
          exp                                               link
0  2018-10-01  https://www.nasdaq.com/symbol/t/option-chain?d...
1  2018-11-01  https://www.nasdaq.com/symbol/t/option-chain?d...
2  2018-12-01  https://www.nasdaq.com/symbol/t/option-chain?d...
3  2019-01-01  https://www.nasdaq.com/symbol/t/option-chain?d...
4  2019-04-01  https://www.nasdaq.com/symbol/t/option-chain?d...
5  2019-06-01  https://www.nasdaq.com/symbol/t/option-chain?d...
6  2020-01-01  https://www.nasdaq.com/symbol/t/option-chain?d...
7  2021-01-01  https://www.nasdaq.com/symbol/t/option-chain?d...
Selected: 2018-11-01
Loading: https://www.nasdaq.com/symbol/t/option-chain?dateindex=2
Parsed columns: ['Calls', 'Last', 'Chg', 'Bid', 'Ask', 'Vol', 'Open', 'Root', 'Strike', 'Puts', 'Last', 'Chg', 'Bid', 'Ask', 'Vol', 'Open']
Found 11 OTM calls
Best option for T: Calls                                              Nov 16, 2018
Calls_Ask                       

,Change,Curr R,Debt/Eq,Dividend,Earnings,Gross M,LTDebt/Eq,Market Cap,No.,Oper M,...,ROI,Volume,Ex-Div Date,Dividend Quarterly Cash,Ex-Div Date Est.,Earnings Est.,Call_Strike,Call_Expiration,Call_OpenInt,Call_PriceLast
Ticker,,,,,,,,,,,,,,,,,,,,,
T,0.30%,0.80,1.04,5.93%,Oct 23/a,51.60%,0.92,245.46B,1,12.30%,...,5.00%,"33,644,591",2018-07-09,0.500,2018-10-09,2018-10-23,35.0,2018-11-16,709,0.34
RDS-A,1.91%,1.30,0.41,5.70%,Oct 25/b,17.80%,0.36,275.37B,2,7.20%,...,4.30%,"3,134,687",2018-08-09,0.799,2018-11-09,2018-10-25,70.0,2019-01-18,3767,1.07
VZ,-0.57%,1.00,2.20,4.43%,Oct 23/b,58.00%,2.10,226.34B,3,19.00%,...,11.60%,"13,394,157",2018-10-09,0.603,2018-10-09,2018-10-23,55.0,2018-11-02,715,0.97
XOM,0.26%,0.80,0.22,3.92%,Oct 26/b,32.70%,0.11,357.18B,4,6.00%,...,3.20%,"8,831,357",2018-08-10,0.820,2018-11-10,2018-10-26,90.0,2019-01-18,35344,0.84
CVX,0.53%,1.10,0.25,3.79%,Oct 26/b,41.70%,0.20,224.27B,5,7.00%,...,1.70%,"4,767,151",2018-08-16,1.120,2018-11-16,2018-10-26,120.0,2018-12-21,3087,3.60
NVS,-0.12%,1.10,0.42,3.50%,Oct 17/b,67.00%,0.30,215.36B,6,29.00%,...,7.10%,"1,878,984",2018-03-06,1.909,2019-03-06,2018-10-17,90.0,2019-04-18,53,2.05
PG,-0.30%,0.80,0.61,3.42%,Jul 31/b,50.00%,0.41,210.85B,7,20.50%,...,13.10%,"5,925,207",2018-07-19,0.717,2018-10-19,2018-10-31,87.5,2018-11-16,1183,0.57
WFC,0.35%,-,1.22,3.16%,Oct 12/b,-,1.22,263.77B,8,77.50%,...,9.80%,"21,841,850",2018-08-09,0.430,2018-11-09,2018-10-12,55.0,2019-01-18,39815,2.26
PFE,1.16%,1.20,0.58,3.13%,Jul 31/b,78.90%,0.41,255.23B,9,25.40%,...,11.00%,"19,248,793",2018-08-02,0.340,2018-11-02,2018-10-31,44.0,2018-12-21,4172,1.05


In [254]:
perf = screener_df[["Price"]].astype(float).copy()
perf["Ex-Div Date Est."] = screener_df["Ex-Div Date Est."]
perf["Dividend Credit"] = screener_df["Dividend Quarterly Cash"]
perf["Strike"] = screener_df["Call_Strike"]
perf["Expiration"] = screener_df["Call_Expiration"]
perf["Call Credit"] = screener_df["Call_PriceLast"]
perf["Margin Req"] = (perf["Price"] - perf["Call Credit"]) * 100
perf["Credit %"] = (perf["Dividend Credit"] + perf["Call Credit"]) / perf["Price"] * 100
perf["Upside %"] = (perf["Strike"] - perf["Price"]) / perf["Price"] * 100
perf["DTE"] = perf["Expiration"] - datetime.date.today()
perf["Early Exercise Days"] = perf["Expiration"] - perf["Ex-Div Date Est."]

perf

,Price,Ex-Div Date Est.,Dividend Credit,Strike,Expiration,Call Credit,Margin Req,Credit %,Upside %,DTE,Early Exercise Days
Ticker,,,,,,,,,,,
T,33.72,2018-10-09,0.500,35.0,2018-11-16,0.34,3338,2.491103,3.795967,58 days,38 days
RDS-A,66.02,2018-11-09,0.799,70.0,2019-01-18,1.07,6495,2.830960,6.028476,121 days,70 days
VZ,54.39,2018-10-09,0.603,55.0,2018-11-02,0.97,5342,2.892076,1.121530,44 days,24 days
XOM,83.63,2018-11-10,0.820,90.0,2019-01-18,0.84,8279,1.984934,7.616884,121 days,69 days
CVX,118.13,2018-11-16,1.120,120.0,2018-12-21,3.60,11453,3.995598,1.583002,93 days,35 days
NVS,85.03,2019-03-06,1.909,90.0,2019-04-18,2.05,8298,4.656004,5.844996,211 days,43 days
PG,84.00,2018-10-19,0.717,87.5,2018-11-16,0.57,8343,1.532143,4.166667,58 days,28 days
WFC,54.50,2018-11-09,0.430,55.0,2019-01-18,2.26,5224,4.935780,0.917431,121 days,70 days
PFE,43.51,2018-11-02,0.340,44.0,2018-12-21,1.05,4246,3.194668,1.126178,93 days,49 days


In [255]:
perf.to_csv("covered_dividends.csv")